# Keras Neural Network Model
---

In [41]:
import pandas as pd

# read in files as dfs
mpls_df = pd.read_csv('resources/mpls_solar_weather.csv')
olg_df = pd.read_csv('resources/olg_solar_weather.csv')

In [42]:
# mpls_df = mpls_df[mpls_df.power_delivered > 0]

# add column that is 1 if power was delivered and 0 if not
mpls_df['has_power'] = (mpls_df['power_delivered'] > 0).astype(int)
olg_df['has_power'] = (olg_df['power_delivered'] > 0).astype(int)

In [43]:
# columns on which to run ML model
features = [
    'clouds_all', 'temp_f', 'pressure', 'humidity', 'wind_speed', 'wind_deg',
    'rain_1h', 'snow_1h', 'sin_hour', 'cos_hour', 'sin_day', 'cos_day'
]
target = 'has_power'

## Keras Neural Network

In [44]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [45]:
# define target and features
X = mpls_df[features].copy()
y = mpls_df[target]

# X = olg_df[features].copy()
# y = olg_df[target]

# split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [46]:
# scale data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\Jackmarius\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [47]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [48]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [49]:
model = Sequential()

# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=12))
model.add(Dense(units=2, activation='softmax'))

In [50]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               1300      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 202       
Total params: 1,502
Trainable params: 1,502
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=30,
    shuffle=True,
    verbose=2
)

Train on 14894 samples
Epoch 1/30
14894/14894 - 2s - loss: 0.2899 - accuracy: 0.9109
Epoch 2/30
14894/14894 - 1s - loss: 0.1650 - accuracy: 0.9435
Epoch 3/30
14894/14894 - 1s - loss: 0.1529 - accuracy: 0.9473
Epoch 4/30
14894/14894 - 1s - loss: 0.1449 - accuracy: 0.9484
Epoch 5/30
14894/14894 - 1s - loss: 0.1396 - accuracy: 0.9487
Epoch 6/30
14894/14894 - 1s - loss: 0.1353 - accuracy: 0.9492
Epoch 7/30
14894/14894 - 1s - loss: 0.1320 - accuracy: 0.9499
Epoch 8/30
14894/14894 - 1s - loss: 0.1282 - accuracy: 0.9503
Epoch 9/30
14894/14894 - 1s - loss: 0.1266 - accuracy: 0.9516
Epoch 10/30
14894/14894 - 1s - loss: 0.1235 - accuracy: 0.9530
Epoch 11/30
14894/14894 - 1s - loss: 0.1215 - accuracy: 0.9517
Epoch 12/30
14894/14894 - 1s - loss: 0.1193 - accuracy: 0.9539
Epoch 13/30
14894/14894 - 1s - loss: 0.1177 - accuracy: 0.9544
Epoch 14/30
14894/14894 - 1s - loss: 0.1163 - accuracy: 0.9551
Epoch 15/30
14894/14894 - 1s - loss: 0.1161 - accuracy: 0.9541
Epoch 16/30
14894/14894 - 1s - loss: 0.11

In [52]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

4965/1 - 0s - loss: 0.0517 - accuracy: 0.9623
Normal Neural Network - Loss: 0.09791430614884916, Accuracy: 0.9623363614082336


In [53]:
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

# print(f"Predicted classes: {prediction_labels}")
# print(f"Actual Labels: {list(y_test[:50])}")

In [54]:
# save model
# model.save('mpls_nn_YorN.h5')
# model.save('olg_nn_YorN.h5')

In [55]:
predictions = pd.DataFrame(prediction_labels, y_test, )

In [56]:
predictions

,0
has_power,
0,0
1,1
1,1
1,1
1,1
1,1
1,1
0,0
1,1
